# Convertor for ".mphtxt" to hdf5 format for Matlab

In [52]:
from exporter import extract_comsol_matrices_from_file, select_matrix_section

### For P2, all important data are here:

In [53]:
filename = "data/hetero_paper_corrected_p2.mphtxt"
# Example usage:
sections = extract_comsol_matrices_from_file(filename)

print("-----------------------------------")
coord_dict = select_matrix_section(sections, "coordinates", ncol=3)
print(coord_dict["description"])
print(coord_dict["matrix"].shape)

print("-----------------------------------")
elem_dict = select_matrix_section(sections, "elements", ncol=10)
print(elem_dict["description"])
print(elem_dict["matrix"].shape)

print("-----------------------------------")
elem_tags_dict = select_matrix_section(sections, "indices", nrow=elem_dict["matrix"].shape[0])
print(elem_tags_dict["description"])
print(elem_tags_dict["matrix"].shape)

print("-----------------------------------")
face_dict = select_matrix_section(sections, "elements", ncol=6)
print(face_dict["description"])
print(face_dict["matrix"].shape)

print("-----------------------------------")
face_tags_dict = select_matrix_section(sections, "indices", nrow=face_dict["matrix"].shape[0])
print(face_tags_dict["description"])
print(face_tags_dict["matrix"].shape)

-----------------------------------
Mesh vertex coordinates
(19211, 3)
-----------------------------------
Type #3
4 tet2 # type name
10 # number of vertices per element
12676 # number of elements
Elements
(12676, 10)
-----------------------------------
12676 # number of geometric entity indices
Geometric entity indices
(12676, 1)
-----------------------------------
Type #2
4 tri2 # type name
6 # number of vertices per element
4257 # number of elements
Elements
(4257, 6)
-----------------------------------
4257 # number of geometric entity indices
Geometric entity indices
(4257, 1)


### For P1 (just the mesh), all important data are here:

In [54]:
filename = "data/hetero_paper_corrected_p1.mphtxt"
# Example usage:
sections = extract_comsol_matrices_from_file(filename)

print("-----------------------------------")
coord_dict = select_matrix_section(sections, "coordinates", ncol=3)
print(coord_dict["description"])
print(coord_dict["matrix"].shape)

print("-----------------------------------")
elem_dict = select_matrix_section(sections, "elements", ncol=4)
print(elem_dict["description"])
print(elem_dict["matrix"].shape)

print("-----------------------------------")
elem_tags_dict = select_matrix_section(sections, "indices", nrow=elem_dict["matrix"].shape[0])
print(elem_tags_dict["description"])
print(elem_tags_dict["matrix"].shape)

print("-----------------------------------")
face_dict = select_matrix_section(sections, "elements", ncol=3)
print(face_dict["description"])
print(face_dict["matrix"].shape)

print("-----------------------------------")
face_tags_dict = select_matrix_section(sections, "indices", nrow=face_dict["matrix"].shape[0])
print(face_tags_dict["description"])
print(face_tags_dict["matrix"].shape)

-----------------------------------
Mesh vertex coordinates
(2714, 3)
-----------------------------------
Type #3
3 tet # type name
4 # number of vertices per element
12676 # number of elements
Elements
(12676, 4)
-----------------------------------
12676 # number of geometric entity indices
Geometric entity indices
(12676, 1)
-----------------------------------
Type #2
3 tri # type name
3 # number of vertices per element
4257 # number of elements
Elements
(4257, 3)
-----------------------------------
4257 # number of geometric entity indices
Geometric entity indices
(4257, 1)


### P1 import into Mesh3D object

In [55]:
from mesh_tools import Mesh3d
import numpy as np

node_X = coord_dict["matrix"][:, 0]
node_Y = coord_dict["matrix"][:, 1]
node_Z = coord_dict["matrix"][:, 2]
elem = elem_dict["matrix"]
face = face_dict["matrix"]

Post processing taken from previous scripts, and **specific for this problem exports**.

In [56]:
mesh = Mesh3d(node_X, node_Y, node_Z, elem, face)
mesh.P1_to_PN(N=2)
node_X = mesh.node_X.reshape(-1, 1)
node_Y = mesh.node_Y.reshape(-1, 1)
node_Z = mesh.node_Z.reshape(-1, 1)

# P2 nodes and elements
node_PN = np.concatenate([node_X, node_Y, node_Z], axis=1)
elem_PN = np.concatenate([mesh.elem_PN[:, :7],
                          mesh.elem_PN[:, 8:],
                          mesh.elem_PN[:, 7].reshape(-1, 1)], axis=1)

# material tags to conform with 4 materials
tags = elem_tags_dict["matrix"]
material = np.zeros_like(tags)
material[tags == 1] = 1
material[tags == 2] = 2
material[tags == 3] = 1
material[tags == 4] = 2
material[tags == 5] = 0
material[tags == 6] = 0
material[tags == 7] = 3
material[tags == 8] = 3

boundary_outer = np.zeros((face.shape[0], 1), dtype=np.int32)
face_X = np.mean(node_X[face], axis=1)
face_Y = np.mean(node_Y[face], axis=1)
face_Z = np.mean(node_Z[face], axis=1)
x_sum = max(node_X)
x_sum_min = min(node_X)
y_sum = max(node_Y)
y_sum_min = min(node_Y)
z_sum = max(node_Z)
z_sum_min = min(node_Z)

boundary_outer[np.isclose(face_X, x_sum_min, atol=1e-2)] = 1
boundary_outer[np.isclose(face_X, x_sum, atol=1e-2)] = 2
boundary_outer[np.isclose(face_Y, y_sum_min, atol=1e-2)] = 3
boundary_outer[np.isclose(face_Y, y_sum, atol=1e-2)] = 4
boundary_outer[np.isclose(face_Z, z_sum_min, atol=1e-2)] = 5
boundary_outer[np.isclose(face_Z, z_sum, atol=1e-2)] = 6

node = coord_dict["matrix"]
face_PN = mesh.face_PN

#### Export to hdf5


In [57]:
import h5py
file_name = "test_michalec_opraveny.h5"

name_all = ['nodeP1', 'node', 'elem', 'material', 'faceP1', 'face', 'boundary']
data_all = [node, node_PN, elem_PN, material, face, mesh.face_PN, boundary_outer]

with h5py.File(file_name, 'w') as f:
    for name, data in zip(name_all, data_all):
        f.create_dataset(name=name, data=data, compression='gzip', compression_opts=9)